### 1.1 The Fibonacci Sequence
---

The fibonacci sequence is a sequence of numbers such that any number, except for the first and the second, is the sum of the previous two: 0, 1, 1, 2, 3, 5, 8, 13, 21, ...

The value of the first Fibonacci number in the sequence is 0. The value of the fourth Fibonacci number is 2. It follows that to get the value of any Fibonacci number, n, in the sequence, one can use the formula. 

fib(n) = fib(n - 1) + fib(n - 2)

#### 1.1.1 A first recursive attempt

The preceding formula for computing a number in the Fibonacci sequence is a form of code that can be trivially translated into a ***recursive*** Python function. (A recursive function is a function that calls itself). This mechanical translation will serve as our first attempt at writing a function to return a given value of the Fibonacci sequence. 


In [1]:
def fib1(n: int) -> int: 
    return fib1(n - 1) + fib1(n - 2)

print(fib1(5))
# this creates an error: max recursion depth

RecursionError: maximum recursion depth exceeded

The issues this creates is that the fib1 function will run forever without returning a final result. Every call to fib1() results in another two calls of fib1() with no end in sight. We call such a circumstance *infinite recursion* and it is analogous to an *infinit loop* 

#### 1.1.2 Utilizing base cases

Notice that until you run fib1(), there is no indication from your Python environment that there is anything wrong. It is the duty of the programmer to avoid infinite recursion. The reason for the infinite recursion is that we never specified a base case. In a recursive function, a base case serves as a stopping point. 

In the case of the Fibonacci function, we have natural base cases in the form of the special first two sequence values, 0, and 1. Neither 0 nor 1 is the sum of the previous two numbers in the sequence. Instead, they are the special first two values. 

In [2]:
def fib2(n: int) -> int:
    if n < 2:    # base case
        return n
    return fib2(n - 2) + fib2(n - 1)

**Note**: The fib2 version of the Fibonacci function returns 0 as the zeroth number, rather than the first number, as in our original proposition. In a programming context, this kind of makes sense because we are used to sequences starting with a zeroth context.

fib2() can be called successfully and will return correct results. Try calling it with some small values. 

In [3]:
print(fib2(5))
print(fib2(10))

5
55


Do not try calling fib2(50). It will never finish executing! Every call results in two more calls to fib2() by way of the recursive calls fib2(n - 1) and fib2(n - 2). In other words, the call tree grows exponentially.

#### 1.1.3 Memorization to the rescue

*Memorization* is a technique in which you store the results of computational tasks when they are completed so that when you need them again, you can look them up instead of needing to compute them a second time. 

Let's create a new version of the Fibonacci function that utilizes a Python dictionary for memorization purposes.

In [4]:
from typing import Dict
memo: Dict[int, int] = {0: 0, 1: 1}    # our base cases
    
def fib3(n: int) -> int:
    if n not in memo:
        memo[n] = fib3(n - 1) + fib3(n - 2)    # memorization
    return memo[n]

print(fib3(5))
print(fib3(50))

5
12586269025


A call to fib3(20) will result in just 39 calls of fib3 as opposed to th 21,891 of fib2 resulting from the call fib(20). memo is prefilled with the earlier base cases of 0 and 1, saving fib3() from the complexity of another if statement. 

#### 1.1.4 Automatic memorization

fib3() can be further simplified. Python has a built-in decorator for memoizing any function automagically. In fib4(), the decorator @functools.lru_cache() is used with the same exact code as we used in fib2(). Each time fib4() is executed with a novel argument, the decorator causes the return value to be cached. Upon future calls of fib4() with the same argument, the previous return value of fib4() for that arguement is retrieved from the cache and returned.

In [5]:
from functools import lru_cache

@lru_cache(maxsize=None)
def fib4(n: int) -> int:    # same definition as fib2()
    if n < 2:    # base case
        return n
    return fib4(n - 2) + fib4(n - 1)    # recursive case

print(fib4(5))
print(fib4(50))

5
12586269025


Note that we can calculate fib4(50) instantly, even though the body of the Fibonacci function is the same as that in fib2().@lru_cache's maxsize property indicates how many of the most recent calls of the function it is decorating should be cached. Setting it to None indicates that there is no limit. 

#### 1.1.5 Keep it simple, Fibonacci

There is an even more performant option. We can solve this with an old-fashioned iterative approach. 

In [6]:
def fib5(n: int) -> int:
    if n == 0: return n    # special case
    last: int = 0    # initially set to fib(0)
    next: int = 1    # initially set to fib(1)
    for _ in range(1, n):
        last, next = next, last + next
    return next

print(fib5(5))
print(fib5(50))

5
12586269025


**Warning**: The body of the for loop in fib5() uses tuple unpacking in perhaps a bit of an overly clever way. Some may feel that it sacrifices readability for conciseness. Others may find the conciseness in and of itself more readable. The gist is, last is being set to the previous value of next, and next is being set to the previous value of last plus the previous value of next. This avoids the creation of a temporary variable to hold the old value of next after last is updated but before next is updated. Using tuple unpacking in this fashion for some kind of variable swap in common in Python. 

With this approach, the body of the for loop will run a maximum of n - 1 times. In other words, this is the most efficient version yet. Compare 19 runs of the for loop body to 21,891 recursive calls of fib2() for the 20th Fibonacci number. That could make a serious difference in real-world applications.
    In the recursive solutions, we worked backward. In this iterative solution, we work forward. Sometimes recursion is the most intuitive way to solve a problem. For example, the meat of fib1() and fib2() is pretty much a mechanical translation of the original Fibonacci formula. However, naive recursive solutions can also come with signigicant performance costs. Remember, any problem that can be solved recursively can also be solved iteratively. 
    
#### 1.1.6 Generating Fibonacci numbers with a generator

So far, we have written functions that output a single value in the Fibonacci sequence. What if we want to output the entire sequence up to some value instead? It is easy to convert fib5() into a Python generator using the yeild statement. When the generator is iterated, each iteration will spew a value from the Fibonacci sequence using a yield statement.

In [7]:
from typing import Generator

def fib6(n: int) -> Generator[int, None, None]:
    yield 0    # special case
    if n > 0: yield 1    # special case
    last: int = 0    # initially set to fib(0)
    next: int = 1    # initially set to fib(1)
    for _ in range(1, n):
        last, next = next, last + next
        yield next    # main generation step
        
for i in fib6(50):
    print(i)

0
1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765
10946
17711
28657
46368
75025
121393
196418
317811
514229
832040
1346269
2178309
3524578
5702887
9227465
14930352
24157817
39088169
63245986
102334155
165580141
267914296
433494437
701408733
1134903170
1836311903
2971215073
4807526976
7778742049
12586269025


If you run fib6.py, you will see 51 numbers in the Fibonacci sequence printed. For each iteration of the for loop for i in fib6(50):, fib6() runs through to a yield statement. If the end of the function is reached and there are no more yield statements, the loop finishes iterating. 